In [1]:
# Import Python wrapper for or-tools CP-SAT solver
from ortools.sat.python import cp_model

In [2]:
# Create the model.
model = cp_model.CpModel()

In [3]:
 jobs_data = [ # task = (machine_id, processing_time).
        [(0, 3), (1, 2), (2, 2)],   # Job0,
        [(0, 2), (2, 1), (1, 4)],  # Job1
        [(1, 4), (2, 3)]           # Job2
    ]

In [4]:
machine_count = 1 + max(task[0] for job in jobs_data for task in job)
all_machines = range(machine_count)

In [5]:
machine_count

3

In [6]:
all_machines

range(0, 3)

In [7]:
# Computes horizon dynamically as the sum of all durations
horizon = sum(task[1] for job in jobs_data for task in job)

In [8]:
horizon

21

In [9]:
import collections

# Named tuple to store information about created variables.
task_type = collections.namedtuple('task_type', 'start end interval')

#Named tuple to manipulate solution information.
assigned_task_type = collections.namedtuple('assigned_task_type', 'start job index duration')


In [10]:
task_type

__main__.task_type

In [11]:
# Creates job intervals and add to the corresponding machine lists.

all_tasks = {}
machine_to_intervals = collections.defaultdict(list)

In [12]:
for job_id, job in enumerate(jobs_data):
    for task_id, task in enumerate(job):
        machine = task[0]
        duration = task[1]
        suffix = '_%i_%i' % (job_id, task_id)
        start_var = model.NewIntVar(0, horizon, 'Start' + suffix)
        end_var = model.NewIntVar(0, horizon, 'end' + suffix)
        interval_var = model.NewIntervalVar(start_var, duration, end_var, 'interval' + suffix)
        all_tasks[job_id, task_id] = task_type( 
                                        start = start_var, end = end_var, interval = interval_var)
        machine_to_intervals[machine].append(interval_var)
       

In [13]:
# Define the constraints
# Create and add disjunctive constraints.
for machine in all_machines:
    model.AddNoOverlap(machine_to_intervals[machine])
    
# Precedences inside a job
for job_id, job in enumerate(jobs_data):
    for task_id in range(len(job) - 1):
        model.Add(all_tasks[job_id, task_id + 1].start >= all_tasks[job_id, task_id].end)

In [14]:
# Define an objective
# Makespan objective

obj_var = model.NewIntVar(0, horizon, 'makespan')
model.AddMaxEquality(obj_var, [ all_tasks[job_id, len(job) - 1].end
                              for job_id, job in enumerate(jobs_data)
                              ])
model.Minimize(obj_var)


In [15]:
# Declares the solver
# Solve model.

solver = cp_model.CpSolver()
status = solver.Solve(model)

In [16]:
# Display the results

# Create one list of assigned tasks per machine.
assigned_jobs = collections.defaultdict(list)
for job_id, job in enumerate(jobs_data):
    for task_id, task in enumerate(job):
        machine = task[0]
        assigned_jobs[machine].append(
            assigned_task_type(
                start = solver.Value(all_tasks[job_id, task_id].start),
                job = job_id,
                index = task_id,
                duration = task[1]))

In [17]:
# Create per machine output lines.

output = ''
for machine in all_machines:
    # Sort by starting time.
    assigned_jobs[machine].sort()
    sol_line_tasks = 'Machine' + str(machine) + str(machine) + ': '
    sol_line = '                 ' 
    
    for assigned_task in assigned_jobs[machine]:
        name = 'job_%i_%i' % (assigned_task.job, assigned_task.index)
        # Add spaces to output to align columns.
        sol_line_tasks += '%-10s' % name
        
        start = assigned_task.start
        duration = assigned_task.duration
        sol_tmp = '[%i, %i]' % (start, start + duration)
        # Add spaces to output to align columns
        sol_line += '%-10s' % sol_tmp
        
    sol_line += '\n'
    sol_line_tasks += '\n'
    output += sol_line_tasks
    output += sol_line
    
# Finally print the solution found

print('Optimal Schedule Length: %i' % solver.ObjectiveValue())
print(output)

Optimal Schedule Length: 11
Machine00: job_0_0   job_1_0   
                 [0, 3]    [3, 5]    
Machine11: job_2_0   job_0_1   job_1_2   
                 [0, 4]    [4, 6]    [7, 11]   
Machine22: job_1_1   job_0_2   job_2_1   
                 [5, 6]    [6, 8]    [8, 11]   



### Complete Program

In [18]:
from __future__ import print_function

import collections

# Import Python wrapper for or-tools CP-SAT solver.
from ortools.sat.python import cp_model


def MinimalJobshopSat():
    """Minimal jobshop problem."""
    # Create the model.
    model = cp_model.CpModel()

    jobs_data = [  # task = (machine_id, processing_time).
        [(0, 3), (1, 2), (2, 2)],  # Job0
        [(0, 2), (2, 1), (1, 4)],  # Job1
        [(1, 4), (2, 3)]  # Job2
    ]

    machines_count = 1 + max(task[0] for job in jobs_data for task in job)
    all_machines = range(machines_count)

    # Computes horizon dynamically as the sum of all durations.
    horizon = sum(task[1] for job in jobs_data for task in job)

    # Named tuple to store information about created variables.
    task_type = collections.namedtuple('task_type', 'start end interval')
    # Named tuple to manipulate solution information.
    assigned_task_type = collections.namedtuple('assigned_task_type',
                                                'start job index duration')

    # Creates job intervals and add to the corresponding machine lists.
    all_tasks = {}
    machine_to_intervals = collections.defaultdict(list)

    for job_id, job in enumerate(jobs_data):
        for task_id, task in enumerate(job):
            machine = task[0]
            duration = task[1]
            suffix = '_%i_%i' % (job_id, task_id)
            start_var = model.NewIntVar(0, horizon, 'start' + suffix)
            end_var = model.NewIntVar(0, horizon, 'end' + suffix)
            interval_var = model.NewIntervalVar(start_var, duration, end_var,
                                                'interval' + suffix)
            all_tasks[job_id, task_id] = task_type(
                start=start_var, end=end_var, interval=interval_var)
            machine_to_intervals[machine].append(interval_var)

    # Create and add disjunctive constraints.
    for machine in all_machines:
        model.AddNoOverlap(machine_to_intervals[machine])

    # Precedences inside a job.
    for job_id, job in enumerate(jobs_data):
        for task_id in range(len(job) - 1):
            model.Add(all_tasks[job_id, task_id +
                                1].start >= all_tasks[job_id, task_id].end)

    # Makespan objective.
    obj_var = model.NewIntVar(0, horizon, 'makespan')
    model.AddMaxEquality(obj_var, [
        all_tasks[job_id, len(job) - 1].end
        for job_id, job in enumerate(jobs_data)
    ])
    model.Minimize(obj_var)

    # Solve model.
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL:
        # Create one list of assigned tasks per machine.
        assigned_jobs = collections.defaultdict(list)
        for job_id, job in enumerate(jobs_data):
            for task_id, task in enumerate(job):
                machine = task[0]
                assigned_jobs[machine].append(
                    assigned_task_type(
                        start=solver.Value(all_tasks[job_id, task_id].start),
                        job=job_id,
                        index=task_id,
                        duration=task[1]))

        # Create per machine output lines.
        output = ''
        for machine in all_machines:
            # Sort by starting time.
            assigned_jobs[machine].sort()
            sol_line_tasks = 'Machine ' + str(machine) + ': '
            sol_line = '           '

            for assigned_task in assigned_jobs[machine]:
                name = 'job_%i_%i' % (assigned_task.job, assigned_task.index)
                # Add spaces to output to align columns.
                sol_line_tasks += '%-10s' % name

                start = assigned_task.start
                duration = assigned_task.duration
                sol_tmp = '[%i,%i]' % (start, start + duration)
                # Add spaces to output to align columns.
                sol_line += '%-10s' % sol_tmp

            sol_line += '\n'
            sol_line_tasks += '\n'
            output += sol_line_tasks
            output += sol_line

        # Finally print the solution found.
        print('Optimal Schedule Length: %i' % solver.ObjectiveValue())
        print(output)
        
        
MinimalJobshopSat()

Optimal Schedule Length: 11
Machine 0: job_0_0   job_1_0   
           [0,3]     [3,5]     
Machine 1: job_2_0   job_0_1   job_1_2   
           [0,4]     [4,6]     [7,11]    
Machine 2: job_1_1   job_0_2   job_2_1   
           [5,6]     [6,8]     [8,11]    

